In [123]:
import pandas as pd
import numpy as np

## Prep Work

Get Games

In [133]:
from nba_api.live.nba.endpoints import scoreboard

# Today's Score Board
games = scoreboard.ScoreBoard()
print('There are',len(games.get_dict().get('scoreboard').get('games')),'games today')

There are 14 games today


Get Game IDs

In [107]:
game_ids = []
for i in range(len(games.get_dict().get('scoreboard').get('games'))):
    game_ids.append(games.get_dict().get('scoreboard').get('games')[i].get('gameId'))
    

## Get Game Box Scores

In [109]:
#weighted mean of 9 Cat from 2021-22 season
weighted_mean = [12.44,4.71,2.81,0.9,0.6,1.4,0.46,0.77,1.46]
#weighted stdev of 9 Cat from 2021-22 season
weighted_StDev = [6.44,2.51,2.08,0.37,0.41,0.92,0.075,0.11,0.9]

In [110]:
from nba_api.stats.endpoints import BoxScoreTraditionalV2
df_list=[]
for i in game_ids:
    # Today's Score Board
    boxscore = BoxScoreTraditionalV2(i)
    # Headers
    header = boxscore.get_dict().get('resultSets')[0].get('headers')
    # dictionary
    df = pd.DataFrame(boxscore.get_dict().get('resultSets')[0].get('rowSet'), columns = header)
    df = df[['PLAYER_NAME','MIN','FGM',
       'FGA', 'FG_PCT', 'FG3M', 'FG3A', 'FG3_PCT', 'FTM', 'FTA', 'FT_PCT',
       'OREB', 'DREB', 'REB', 'AST', 'STL', 'BLK', 'TO', 'PF', 'PTS']]
    df_list.append(df)

box_score = pd.concat(df_list)
box_score_9cat = box_score[['PLAYER_NAME', 'PTS','REB', 'AST', 'STL', 'BLK','FG3M', 'FG_PCT', 'FT_PCT','TO']]

### Get Z-scores

In [117]:
z_list = []
for i in range(len(box_score_9cat)):
    # Get Z-score for each Cat
    diff_from_mean = np.subtract(box_score_9cat.iloc[i].tolist()[1:],weighted_mean)
    stdev_from_mean = np.divide(diff_from_mean,weighted_StDev) #z-score of each category
    # Weight each Cat
        # TO: -0.1
        # FG%, FT%: weight by attempt*0.25
    adjusted_scores = np.multiply(stdev_from_mean,[1,1,1,1,1,1,(box_score.FGA.iloc[i]/10.213),(box_score.FTA.iloc[i]/2.575),-1])
    # Sum all scores
    z_list.append(round(np.sum(adjusted_scores),2))

box_score_9cat['Z_Score'] = z_list
box_score['Z_Score'] = z_list

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]


## Rank

In [135]:
box_score_today = box_score_9cat.sort_values('Z_Score',ascending = False).head(30)
box_score_today

,PLAYER_NAME,PTS,REB,AST,STL,BLK,FG3M,FG_PCT,FT_PCT,TO,Z_Score
4,Tyrese Haliburton,43.0,2.0,7.0,1.0,1.0,10.0,0.700,1.000,3.0,24.89
14,Joel Embiid,44.0,7.0,3.0,1.0,2.0,1.0,0.600,1.000,1.0,20.83
4,Luka Doncic,50.0,8.0,10.0,3.0,0.0,6.0,0.567,0.833,7.0,20.52
16,Shai Gilgeous-Alexander,44.0,10.0,6.0,2.0,1.0,3.0,0.586,0.778,2.0,18.65
15,Al Horford,17.0,11.0,2.0,2.0,3.0,5.0,0.429,0.000,0.0,16.62
14,Aleksej Pokusevski,17.0,6.0,3.0,4.0,1.0,3.0,0.636,0.000,1.0,15.45
4,Damian Lillard,34.0,7.0,8.0,0.0,0.0,4.0,0.455,1.000,0.0,15.28
6,Brandon Clarke,24.0,10.0,0.0,2.0,0.0,0.0,0.700,1.000,1.0,14.30
16,Dejounte Murray,26.0,3.0,5.0,4.0,0.0,3.0,0.556,0.750,1.0,13.62
4,Monte Morris,16.0,2.0,5.0,3.0,0.0,2.0,0.571,1.000,0.0,12.90


In [30]:
box_score_today.to_csv('box_score_today.csv')

## Archive

In [79]:
import datetime
from datetime import date, timedelta, datetime

In [80]:
today = date.today().isoformat()
date = date.today()-timedelta(1)
date = datetime.strftime(date, '%Y-%m-%d')

In [ ]:
box_score['Date']=date
cols = list(box_score.columns)
cols = [cols[-1]]+cols[:-1]
box_score = box_score[cols]

In [ ]:
import os
if not os.path.isfile("box_score_archive_22_23.csv"):
    with open("box_score_archive_22_23.csv", "a") as box_score_archive:
        box_score_archive.close()

In [72]:
box_score_archive = pd.read_csv('box_score_archive_22_23.csv')
box_score_archive = box_score_archive.iloc[: , 1:]
box_score_archive = box_score_archive.append(box_score)
box_score_archive.to_csv('box_score_archive_22_23.csv')